In [241]:
import torch
import torch.nn as nn
from torch.autograd import Variable
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence
import torch.nn.functional as F
import numpy as np
import itertools
import pickle
%autosave 180

Autosaving every 180 seconds


In [242]:
# This will be our input

text = 'MoriMoriMoriMoriMori'

In [243]:
character_list = list(set(text))   # get all of the unique letters in our text variable
vocabulary_size = len(character_list)   # count the number of unique elements
character_dictionary = {'i': 0, 'o': 1, 'r': 2, 'M': 3}
# {char:e for e, char in enumerate(character_list)}  # create a dictionary mapping each unique char to a number
encoded_chars = [character_dictionary[char] for char in text] #integer representation of our vocabulary 

In [244]:
print(character_dictionary)

{'i': 0, 'o': 1, 'r': 2, 'M': 3}


In [245]:
print(encoded_chars)

[3, 1, 2, 0, 3, 1, 2, 0, 3, 1, 2, 0, 3, 1, 2, 0, 3, 1, 2, 0]


In [246]:
def one_hot_encode(encoded, vocab_size):
    result = torch.zeros((len(encoded), vocab_size))
    for i, idx in enumerate(encoded):
        result[i, idx] = 1.0
    return result

In [247]:
# One hot encode our encoded charactes
batch_size = 2
seq_length = 3
num_samples = (len(encoded_chars) - 1) // seq_length # time lag of 1 for creating the labels
vocab_size = 4

data = one_hot_encode(encoded_chars[:seq_length*num_samples], vocab_size).reshape((num_samples, seq_length, vocab_size))
num_batches = len(data) // batch_size
X = data[:num_batches*batch_size].reshape((num_batches, batch_size, seq_length, vocab_size))
# swap batch_size and seq_length axis to make later access easier
X = X.transpose(1, 2)

In [ ]:
labels = one_hot_encode(encoded_chars[1:seq_length*num_samples+1], vocab_size) 
y = labels.reshape((num_batches, batch_size, seq_length, vocab_size))
y = y.transpose(1, 2) 
y,y.shape

In [249]:
torch.manual_seed(1) # reproducibility

####  Define the network parameters:
hiddenSize = 2 # network size, this can be any number (depending on your task)
numClass = 4 # this is the same as our vocab_size

#### Weight matrices for our inputs 
Wz = Variable(torch.randn(vocab_size, hiddenSize), requires_grad=False)
Wr = Variable(torch.randn(vocab_size, hiddenSize), requires_grad=False)
Wh = Variable(torch.randn(vocab_size, hiddenSize), requires_grad=False)

## Intialize the hidden state
# this is for demonstration purposes only, in the actual model it will be initiated during training a loop over the 
# the number of bacthes and updated before passing to the next GRU cell.
h_t_demo = torch.zeros(batch_size, hiddenSize) 

#### Weight matrices for our hidden layer
Uz = Variable(torch.randn(hiddenSize, hiddenSize), requires_grad=False)
Ur = Variable(torch.randn(hiddenSize, hiddenSize), requires_grad=False)
Uh = Variable(torch.randn(hiddenSize, hiddenSize), requires_grad=False)

#### bias vectors for our hidden layer
bz = Variable(torch.zeros(hiddenSize), requires_grad=False)
br = Variable(torch.zeros(hiddenSize), requires_grad=False)
bh = Variable(torch.zeros(hiddenSize), requires_grad=False)

#### Output weights
Wy = Variable(torch.randn(hiddenSize, numClass), requires_grad=False)
by = Variable(torch.zeros(numClass), requires_grad=False)

In [ ]:
# h gets updated and then we calculate for the next 
h_t_1 = []
h = h_t_demo
for i,sequence in enumerate(X[0]):   # iterate over each sequence in the batch to calculate the hidden state h 
    z = torch.sigmoid(torch.matmul(sequence, Wz) + torch.matmul(h, Uz) + bz)
    r = torch.sigmoid(torch.matmul(sequence, Wr) + torch.matmul(h, Ur) + br)
    h_tilde = torch.tanh(torch.matmul(sequence, Wh) + torch.matmul(r * h, Uh) + bh)
    h = z * h + (1 - z) * h_tilde
    h_t_1.append(h)
    print(f'h{i}:{h}')
h_t_1 = torch.stack(h_t_1)

In [251]:
def gru(x, h):
    outputs = []
    for i,sequence in enumerate(x): # iterates over the sequences in each batch
        z = torch.sigmoid(torch.matmul(sequence, Wz) + torch.matmul(h, Uz) + bz)
        r = torch.sigmoid(torch.matmul(sequence, Wr) + torch.matmul(h, Ur) + br)
        h_tilde = torch.tanh(torch.matmul(sequence, Wh) + torch.matmul(r * h, Uh) + bh)
        h = z * h + (1 - z) * h_tilde

        # Linear layer
        y_linear = torch.matmul(h, Wy) + by

        # Softmax activation function
        y_t = F.softmax(y_linear, dim=1)

        outputs.append(y_t)
    return torch.stack(outputs), h
    

In [252]:
def sample(primer, length_chars_predict):
    
    word = primer

    primer_dictionary = [character_dictionary[char] for char in word]
    test_input = one_hot_encode(primer_dictionary, vocab_size)
    

    h = torch.zeros(1, hiddenSize)

    for i in range(length_chars_predict):
        outputs, h = gru(test_input, h)
        choice = np.random.choice(vocab_size, p=outputs[-1][0].numpy())
        word += character_list[choice]
        input_sequence = one_hot_encode([choice],vocab_size)
    return word


In [ ]:
max_epochs = 20  # passes through the data
for e in range(max_epochs):
    h = torch.zeros(batch_size, hiddenSize)
    for i in range(num_batches):
        x_in = X[i]
        y_in = y[i]
        
        out, h = gru(x, h)
        print(sample('Mo',20))